# 2: GAN training component

This handles the training of the GAN deep abstraction method using the datasets generated in 1.

## Step 0: Setup

In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
from simulation_manager import SimulationManager
from gan_manager import GanManager

import pickle
import os
from time import time

In [ ]:
current_dir = os.getcwd()

In [ ]:
def train_per_config(name, path, train_config):
    """
    Train a GAN deep abstract model for each provided case.
    Example `train_config` dict:
        {"case_1": {
                "end_time": 32,
                "n_steps": 16,
                "n_init_conditions": 100,
                "n_sims_per_init_condition": 2000,
                "n_epochs": 1,
                "batch_size": 256,
                "n_critic": 5
            }
        }

    The trained models are stored in the "trained_gan" directory.
    
    Parameters:
    - name: string representing the name of the CRN
    - path: string representing the filepath to the CRN definition (Antimony .txt or SML .xml)
    - train_config: dict containing the different training configurations
    """
    training_times = dict()
    if not os.path.exists("trained_gan"):
        os.makedirs("trained_gan")
        
    for case in train_config:
        config = train_config[case]
        end_time = config["end_time"]
        n_steps = config["n_steps"]
        n_init_conditions = config["n_init_conditions"]
        n_sims_per_init_condition = config["n_sims_per_init_condition"]
        n_epochs = config["n_epochs"]
        batch_size = config["batch_size"]
        n_critic = config["n_critic"]

        config_name = f"{name}_{n_steps}_{end_time}_{n_init_conditions}_{n_sims_per_init_condition}"
    
        with open(f"ssa_datasets/{config_name}_train.pickle", "rb") as f:
            training_data = pickle.load(f)

        with open(f"ssa_datasets/{config_name}_test.pickle", "rb") as f:
            test_data = pickle.load(f)
    
        sm = SimulationManager(
            path_to_sbml=path,
            model_name=name,
            n_init_conditions=n_init_conditions,
            n_sims_per_init_condition=n_sims_per_init_condition,
            end_time=end_time,
            n_steps=n_steps
        )
    
        gm = GanManager(
            model_name=sm.model_name,
            n_epochs=n_epochs,
            batch_size=batch_size,
            species_names=sm.get_species_names(),
            x_dim=sm.get_num_species(),
            traj_len=n_steps,
            end_time=end_time,
            id=config_name
        )
    
        gan_train_data = sm.transform_data_for_gan(training_data)
        gan_test_data = sm.transform_validation_data_for_gan(test_data, n_sims_per_init_condition)
    
        gm.init_dataset(
            train_data=gan_train_data,
            test_data=gan_test_data
        )
        gm.init_models()
    
        start_time = time()
        gm.train(n_critic=n_critic)
        time_taken = time() - start_time
        training_times[case] = time_taken

        print(f"Finished training GAN for {case}.")
    
    with open(f"{name}_gan_training_times.pickle", "wb") as f:
        pickle.dump(training_times, f)

## Step 1: Multifeedback model

In [ ]:
relative_path = "crn_models/1_multifeedback.txt"
path = os.path.join(current_dir, relative_path)
name = "multifeedback"

In [ ]:
train_config = {
    "case_1": {
        "end_time": 32,
        "n_steps": 16,
        "n_init_conditions": 100,
        "n_sims_per_init_condition": 200,
        "n_epochs": 2,
        "batch_size": 256,
        "n_critic": 5
    },
    "case_2": {
        "end_time": 32,
        "n_steps": 16,
        "n_init_conditions": 200,
        "n_sims_per_init_condition": 100,
        "n_epochs": 2,
        "batch_size": 256,
        "n_critic": 5
    }
}

In [ ]:
train_per_config(name, path, train_config)

## Step 2: Repressilator model

In [ ]:
relative_path = "crn_models/2_repressilator.txt"
path = os.path.join(current_dir, relative_path)
name = "repressilator"

In [ ]:
train_config = {
    "case_1": {
        "end_time": 128,
        "n_steps": 32,
        "n_init_conditions": 200,
        "n_sims_per_init_condition": 100,
        "n_epochs": 2,
        "batch_size": 256,
        "n_critic": 5
    },
}

In [ ]:
train_per_config(name, path, train_config)